In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, 3, padding=1) # input channel 1, output channel 10
        self.conv2 = nn.Conv2d(10, 20, 3, padding=1) # input channel 10, output channel 20
        self.fc1 = nn.Linear(20*7*7, 50) # Fully connected layer, 50 nodes
        self.fc2 = nn.Linear(50, 10) # Output layer, 10 nodes

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2) # Max pooling over 2x2
        x = F.max_pool2d(F.relu(self.conv2(x)), 2) # Max pooling over 2x2
        x = x.view(-1, 20*7*7) # Reshape before fully connected layer
        x = F.relu(self.fc1(x)) # Fully connected layer with ReLU activation
        x = self.fc2(x) # Output layer
        return F.log_softmax(x, dim=1)


In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 28, 28]             100
            Conv2d-2           [-1, 20, 14, 14]           1,820
            Linear-3                   [-1, 50]          49,050
            Linear-4                   [-1, 10]             510
Total params: 51,480
Trainable params: 51,480
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.09
Params size (MB): 0.20
Estimated Total Size (MB): 0.29
----------------------------------------------------------------


In [4]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [5]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.08867933601140976 batch_id=468: 100%|██████████| 469/469 [00:23<00:00, 20.01it/s]



Test set: Average loss: 0.0853, Accuracy: 9727/10000 (97.2700%)



loss=0.09634191542863846 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 25.13it/s]



Test set: Average loss: 0.0601, Accuracy: 9806/10000 (98.0600%)



loss=0.03172298148274422 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.49it/s]


...


Test set: Average loss: 0.0352, Accuracy: 9914/10000 (99.1400%)



loss=0.0001983097317861393 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 25.01it/s]



Test set: Average loss: 0.0341, Accuracy: 9909/10000 (99.0900%)



loss=0.004613622557371855 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 24.67it/s]



Test set: Average loss: 0.0382, Accuracy: 9904/10000 (99.0400%)

